# Subsetting data by Hex ID

This notebook explores the ability to do fast spatial subsets as H3 Hex operations.  We may also benchmark these against spatial filters. Note this uses some small helper methods from our [boettiger-lab/cng-python](https://github.com/boettiger-lab/cng-python) repo; be sure to have the latest version. 

For the moment this focuses on vector data, though it is of course possible to consider a similar spatial grid approach to rasters as well.  Almost everything here are just simple duckdb operations using the `h3` extension, see [h3-duckdb](https://github.com/isaacbrodsky/h3-duckdb).

In [1]:
# pip install git+https://github.com/boettiger-lab/cng-python

In [2]:
import ibis
from ibis import _
from cng.utils import *
from cng.maps import *
from cng.h3 import *
import os
import re
import leafmap.maplibregl as leafmap

con = ibis.duckdb.connect(extensions = ["spatial", "h3"])
endpoint = os.getenv("AWS_S3_ENDPOINT", "minio.carlboettiger.info")

set_secrets(con)
duckdb_install_h3()

In the [datasets](https://github.com/boettiger-lab/datasets) repo there is code for that generates cloud-native versions of most of our vector datasets.   For many of these, I've computed h3-indexed versions of the data as well, usually at one or more resolutions.  For point data like GBIF, it is natural to do all the resolutions in a single table.  For polygons, we tile the polygons with h3 hexes of varying resolutions, so something like census tracts at zoom 10 needs quite a lot more rows than it does at zoom 8, and it makes sense for maximum performance to have both available as separate files.  

When necessary, we can use `h3_cell_to_parent` or `h3_cell_to_children` method to compute a lower or higher resolution hex from one that we already have, but obviously it is faster if we already have precomputed at the correct resolution. 

**Note** the hex columns are not entirely standardized yet.  for instance `pad` at zoom 10 is stored in nested array format, rather than as one row per hexid.  Note we can unnest if hex cells are in 'array' format, e.g. `mutate(h8 = h3_cell_to_parent(_.h10.unnest(), 8))`, if necessary.  Also ensure that hexid columns share the same name (I usually use the convention that a hex id column at zoom 8 is called `h8`), and that it is a lower-case string (hexes can be stored as strings in upper or lowercase, or as big integers.  Ultimately it may be faster to use the integer format, though some tools like maplibre will need this cast to strings to render).  


For this example we will subset the Protected Areas Data using the 2022 Census Tracts data, both at zoom-8 resolution:

In [3]:
zoom = 8
pad =  con.read_parquet(f"s3://public-biodiversity/pad-us-4/pad-h3-z{zoom}.parquet")
tracts = con.read_parquet(f"s3://public-social-vulnerability/2022-tracts-h3-z{zoom}.parquet").mutate(h8 = _.h8.lower())


It is very fast to filter the PAD data to any arbitrary selection of State(s), Counties, or Tracts from the census data this way.  (While for a single polygon this may not always be faster than a spatially explicit filter, this can be very useful for large-scale joins).  

In [4]:
# here we go: do subset:

aoi = tracts.filter(_.STATE.isin(["Arizona", "New Mexico"])).join(pad, "h8")



In [5]:
# join hex version to the original pad_parquet that has all the columns 
pad_full =  con.read_parquet(f"s3://public-biodiversity/pad-us-4/pad-us-4.parquet")


aoi_pad = aoi.join(pad_full, "row_n").filter(_.GAP_Sts.isin(["1", "2"]))




## Subsetting and Visualizing 

Note that the hexid versions of the datasets do not (always) include all the columns of the original dataset, as they are intended for filtering.  It is usually pretty quick to join them against the original tables to get all the other columns. 


For drawing on a map, there are a few possible strategies that should scale well: 


- We could now filter the PMTiles for those matching the ID,
- or we can plot the hexes directly.


In [6]:
# Filter PMTiles

row_ids = aoi_pad.select('row_n').distinct().execute().iloc[:, 0].tolist()

def get_filter(column, values):
    return ["in", ["get", column], ["literal", values]]
    #return ["all", ["match", ["get", column], values, True, False]]



pmtiles = f"https://{endpoint}/public-biodiversity/pad-us-4/pad-us-4.pmtiles"

style = {
        "version": 8,
        "sources": {"pmtiles": {"type": "vector", "url": f"pmtiles://{pmtiles}"}},
        "layers": [
            {
                "id": "pad",
                "source": "pmtiles",
                "source-layer": "padus4",
                "type": "fill",
                "filter": get_filter("row_n", row_ids),
                "paint": {"fill-color": "blue", "fill-opacity": 0.7},
            }
        ],
    }


m = leafmap.Map(style=terrain_style())
m.add_pmtiles(url=pmtiles, style=style, fit_bounds=True)
m


Map(height='600px', map_options={'bearing': 0, 'center': (0, 20), 'pitch': 0, 'style': {'version': 8, 'sources…

## Plotting as Hexes

Or we can plot the hexes directly instead of PMTiles polygons.  To do this, we can write them directly to a bucket as JSON. While not as performant in rendering as PMTiles, the JSON hexes are relatively fast to write:


In [7]:
path = "s3://public-data/cache/map/ex.json"
to_json(aoi_pad.select("GAP_Sts", "h8").rename(h3id = "h8"), path)

# turn s3 path to URL
url = re.sub("s3://", f"https://{endpoint}/", path)

In [8]:
from cng.maps import *

url = 'https://minio.carlboettiger.info/public-data/cache/map/ex.json'

layer = HexagonLayer(url, 1)

m = leafmap.Map(style=terrain_style())
m.add_deck_layers([layer])
m

Map(height='600px', map_options={'bearing': 0, 'center': (0, 20), 'pitch': 0, 'style': {'version': 8, 'sources…